In [70]:
import fitz  # PyMuPDF
import re
import os

# --- CONFIGURACIÓN ---
DIRECTORIO_PDFS = "."
DIRECTORIO_IMAGENES = "imagenes_capturadas"
DIRECTORIO_SALIDA = "Compilados_PDF" # <--- Nueva línea: Carpeta para los resultados

# La materia a buscar y el nombre del archivo final ahora se usan en la celda 5
# MATERIA_A_BUSCAR = "MAT1620"
# NOMBRE_ARCHIVO_FINAL = f"Compilado_{MATERIA_A_BUSCAR}_Definitivo.pdf"

In [67]:
def encontrar_pagina_respuestas(doc):
    """Busca la página donde comienza la sección de respuestas."""
    for i, page in enumerate(doc):
        if page.search_for("Respuestas", quads=False):
            print(f"'{doc.name}': Sección de respuestas encontrada en pág. {i + 1}.")
            return i
    # Si no encuentra "Respuestas", puede ser que no haya o esté en un formato de imagen.
    # Como plan B, podemos asumir que las últimas ~15% de las páginas son respuestas si el documento es largo.
    if len(doc) > 20:
        limite_aproximado = int(len(doc) * 0.85)
        print(f"'{doc.name}': No se encontró 'Respuestas'. Asumiendo límite en pág. {limite_aproximado}.")
        return limite_aproximado
    return len(doc)

def capturar_preguntas_definitivo(directorio, dir_imagenes):
    os.makedirs(dir_imagenes, exist_ok=True)
    archivos_pdf = [f for f in os.listdir(directorio) if f.startswith('Guia') and f.endswith('.pdf')]
    mapa_preguntas = []
    preguntas_finales = []

    print("--- PASO 1: Mapeando PREGUNTAS con alta precisión... ---")
    docs_abiertos = {nombre: fitz.open(os.path.join(directorio, nombre)) for nombre in archivos_pdf}

    for nombre_archivo, doc in docs_abiertos.items():
        pagina_limite = encontrar_pagina_respuestas(doc)
        for num_pagina in range(pagina_limite):
            pagina = doc[num_pagina]
            instancias = pagina.search_for("Pregunta N°")
            for inst in instancias:
                mapa_preguntas.append({"fuente": nombre_archivo, "pagina": num_pagina, "bbox_inicio": inst})

    mapa_preguntas.sort(key=lambda p: (p["fuente"], p["pagina"], p["bbox_inicio"].y0))
    print(f"Mapeo completado. Se encontraron {len(mapa_preguntas)} preguntas.")
    
    print("\n--- PASO 2: Calculando caja inteligente y capturando... ---")
    for i, p_actual in enumerate(mapa_preguntas):
        doc = docs_abiertos[p_actual["fuente"]]
        pagina = doc[p_actual["pagina"]]
        
        # Área de búsqueda inicial (desde el inicio preciso de esta pregunta hasta la siguiente)
        area_busqueda = fitz.Rect(0, p_actual["bbox_inicio"].y0, pagina.rect.width, pagina.rect.height)
        if i + 1 < len(mapa_preguntas) and mapa_preguntas[i+1]["fuente"] == p_actual["fuente"] and mapa_preguntas[i+1]["pagina"] == p_actual["pagina"]:
            area_busqueda.y1 = mapa_preguntas[i+1]["bbox_inicio"].y0
        
        # Encontrar todos los elementos dentro del área de búsqueda
        caja_contenedora = fitz.Rect(p_actual["bbox_inicio"]) # Iniciar con el bbox del título
        elementos_texto = pagina.get_text("blocks", clip=area_busqueda)
        elementos_imagen = [pagina.get_image_bbox(img) for img in pagina.get_images(full=True) if pagina.get_image_bbox(img).intersects(area_busqueda)]
        
        # Calcular la caja que une a TODOS los elementos
        for block in elementos_texto:
            caja_contenedora.include_rect(fitz.Rect(block[:4]))
        for img_bbox in elementos_imagen:
            caja_contenedora.include_rect(img_bbox)

        # Extraer metadata para nombrar y filtrar
        texto_completo = pagina.get_text("text", clip=caja_contenedora)
        num_match = re.search(r"Pregunta N°\s?(\d+)", texto_completo)
        numero = num_match.group(1) if num_match else f"idx_{i}"
        materia_match = re.search(r"([A-Z]{3,7}\d{3,4})", texto_completo)
        materia = materia_match.group(1) if materia_match else "Sin Materia"
        
        # Capturar la caja final con un pequeño margen estético
        caja_captura_final = caja_contenedora + (-10, -10, 10, 10)
        pix = pagina.get_pixmap(clip=caja_captura_final, dpi=200)
        nombre_imagen = f"{os.path.splitext(p_actual['fuente'])[0]}_P{numero}.png"
        ruta_guardado = os.path.join(dir_imagenes, nombre_imagen)
        pix.save(ruta_guardado)

        preguntas_finales.append({
            "fuente": p_actual["fuente"], "numero": numero, "materia": materia,
            "ruta_imagen": ruta_guardado
        })
        
    for doc in docs_abiertos.values(): doc.close()
    return preguntas_finales

def crear_pdf_hibrido(preguntas_db, materia_filtro, nombre_archivo):
    font_regular = "C:/Windows/Fonts/calibri.ttf"
    font_bold = "C:/Windows/Fonts/calibrib.ttf"

    preguntas_filtradas = sorted([p for p in preguntas_db if p['materia'] == materia_filtro], key=lambda x: (x['fuente'], int(x['numero'])))
    if not preguntas_filtradas:
        print(f"\n❌ No se encontraron preguntas para la materia '{materia_filtro}'.")
        return

    print(f"\n--- PASO 3: Creando PDF Híbrido para '{materia_filtro}' con {len(preguntas_filtradas)} preguntas... ---")
    doc = fitz.open()

    for pregunta in preguntas_filtradas:
        ruta_imagen = pregunta["ruta_imagen"]
        if os.path.exists(ruta_imagen):
            with fitz.open(ruta_imagen) as img_doc:
                img_rect = img_doc[0].rect
                page_height = img_rect.height + 80 # Espacio para el título
                page_width = img_rect.width + 40
                
                page = doc.new_page(width=page_width, height=page_height)
                
                header = f"Pregunta {pregunta['numero']} (Materia: {pregunta['materia']}) | Fuente: {pregunta['fuente']}"
                
                # --- LÍNEA CORREGIDA ---
                # Se reemplazó fontname="helv-b" por fontfile=font_bold.
                page.insert_textbox(fitz.Rect(20, 15, page_width - 20, 55), header, fontfile=font_bold, fontsize=11, color=(0.1, 0.1, 0.1))
                # -----------------------

                page.insert_image(fitz.Rect(20, 60, page_width - 20, page_height - 20), filename=ruta_imagen)
    
    doc.save(nombre_archivo, garbage=4, deflate=True, clean=True)
    print(f"✅ ¡Éxito! Se ha creado el archivo: '{nombre_archivo}'")
    doc.close()

In [68]:
lista_de_preguntas = capturar_preguntas_definitivo(DIRECTORIO_PDFS, DIRECTORIO_IMAGENES)

if lista_de_preguntas:
    print(f"\n✅ PASOS 1 y 2 COMPLETADOS: Se procesaron y capturaron {len(lista_de_preguntas)} preguntas.")
else:
    print("\n❌ FALLÓ LA EXTRACCIÓN: No se encontraron preguntas.")

--- PASO 1: Mapeando PREGUNTAS con alta precisión... ---
'.\Guia de Ejercicios ECF 1_2016.pdf': Sección de respuestas encontrada en pág. 22.
'.\Guia de Ejercicios ECF 1_2017.pdf': Sección de respuestas encontrada en pág. 35.
'.\Guia de Ejercicios ECF 1_2018.pdf': Sección de respuestas encontrada en pág. 29.
'.\Guia de Ejercicios ECF 1_2019.pdf': Sección de respuestas encontrada en pág. 26.
'.\Guia de Ejercicios ECF 2_2015.pdf': Sección de respuestas encontrada en pág. 24.
'.\Guia de Ejercicios ECF 2_2016.pdf': Sección de respuestas encontrada en pág. 40.
'.\Guia de Ejercicios ECF 2_2017.pdf': Sección de respuestas encontrada en pág. 27.
'.\Guia de Ejercicios ECF 2_2018.pdf': Sección de respuestas encontrada en pág. 25.
'.\Guia de Ejercicios ECF 2_2019.pdf': Sección de respuestas encontrada en pág. 26.
'.\Guia de Ejercicios ECF 2_2023.pdf': Sección de respuestas encontrada en pág. 40.
'.\Guia de Ejercicios ECF 2_2024.pdf': Sección de respuestas encontrada en pág. 41.
Mapeo completado. S

In [69]:
if 'lista_de_preguntas' in locals() and lista_de_preguntas:
    crear_pdf_hibrido(lista_de_preguntas, MATERIA_A_BUSCAR, NOMBRE_ARCHIVO_FINAL)
else:
    print("No se puede generar el PDF porque la lista de preguntas está vacía.")


--- PASO 3: Creando PDF Híbrido para 'MAT1620' con 5 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilado_MAT1620_FINAL.pdf'


In [71]:
# Lista de todas las siglas de las materias que queremos procesar
todas_las_materias = [
    "MAT1610", "MAT1620", "MAT1630", "MAT1640", "MAT1203", "EYP1113",
    "FIS1514", "FIS1533", "QIM100E", "FIS1523", "ICS1513", "IIC1103",
    "FIL188"
]

print(f"🚀 Iniciando la creación de {len(todas_las_materias)} compilados en PDF...")

if 'lista_de_preguntas' in locals() and lista_de_preguntas:
    # --- Cambio: Crear la carpeta de salida ---
    os.makedirs(DIRECTORIO_SALIDA, exist_ok=True)
    
    # Contar cuántas preguntas se encontraron por materia
    conteo_por_materia = {materia: 0 for materia in todas_las_materias}
    for pregunta in lista_de_preguntas:
        if pregunta['materia'] in conteo_por_materia:
            conteo_por_materia[pregunta['materia']] += 1

    # Crear los PDFs
    for materia_actual in todas_las_materias:
        if conteo_por_materia[materia_actual] > 0:
            # --- Cambio: Usar la nueva carpeta en el nombre del archivo ---
            nombre_archivo_materia = os.path.join(DIRECTORIO_SALIDA, f"Compilado_{materia_actual}.pdf")
            
            crear_pdf_hibrido(lista_de_preguntas, materia_actual, nombre_archivo_materia)
        else:
            print(f"\n🟡 No se encontraron preguntas para la materia '{materia_actual}'. No se creará el PDF.")
    
    print(f"\n🎉 ¡Proceso completado! Revisa la carpeta '{DIRECTORIO_SALIDA}' para ver tus compilados.")
else:
    print("❌ Error: No se puede iniciar la generación masiva porque la lista de preguntas está vacía. Asegúrate de ejecutar la Celda 3 primero.")

🚀 Iniciando la creación de 13 compilados en PDF...

--- PASO 3: Creando PDF Híbrido para 'MAT1610' con 4 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilados_PDF\Compilado_MAT1610.pdf'

--- PASO 3: Creando PDF Híbrido para 'MAT1620' con 5 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilados_PDF\Compilado_MAT1620.pdf'

--- PASO 3: Creando PDF Híbrido para 'MAT1630' con 5 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilados_PDF\Compilado_MAT1630.pdf'

--- PASO 3: Creando PDF Híbrido para 'MAT1640' con 6 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilados_PDF\Compilado_MAT1640.pdf'

--- PASO 3: Creando PDF Híbrido para 'MAT1203' con 13 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilados_PDF\Compilado_MAT1203.pdf'

--- PASO 3: Creando PDF Híbrido para 'EYP1113' con 18 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilados_PDF\Compilado_EYP1113.pdf'

--- PASO 3: Creando PDF Híbrido para 'FIS1514' con 12 preguntas... ---
✅ ¡Éx